In [1]:
# Ajustar ruta a la raiz del proyecto
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 3
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None
print(f'Old WD: {_oldwd}')
print(f'New WD: {_newwd}')

Old WD: d:/Itzco/Documents/GitHub/BTC-LSTM/src/python/jnotes
New WD: d:/Itzco/Documents/GitHub/BTC-LSTM


In [2]:

from src.python.util.stats import data_transformer
from src.python.util.basic import load_arrow
from src.python.util.stats import histplot
from src.python.util.stats import sample
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np

plt.style.use('ggplot')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#4C72B0'])

# I Descripción de datos

A continuación, se describen las variables de las bases de datos del precio del bitcoin, descargados desde la plataforma Binance:

- **Time**: La marca de tiempo (minutos) que indica el inicio del intervalo en el que se registra la información.

- **Open**: El precio de apertura del Bitcoin en un período de tiempo específico.

- **High**: El precio más alto alcanzado por el Bitcoin en ese mismo período.

- **Low**: El precio más bajo registrado para el Bitcoin durante el mismo período.

- **Close**: El precio de cierre del Bitcoin en el mismo período de tiempo.

- **Volume**: La cantidad total de Bitcoin negociada en el mercado durante el período.

- **VolumeUSDT**: La cantidad total en dólares estadounidenses (USDT) negociada durante el período.

- **TakerVolume**: La cantidad de Bitcoin intercambiada por los tomadores de mercado durante el período de tiempo especificado.

- **TakerVolumeUSDT**: La cantidad en dólares estadounidenses (USDT) intercambiada por los tomadores de mercado durante el mismo período de tiempo.

- **Trades**: El número total de transacciones o operaciones realizadas durante el período especificado.


**Nota**: Los tomadores de mercado son aquellos que toman las ofertas existentes por lo que aportan liquidez al mercado.

In [3]:
# carga de datos descargados en termporalidad de "30m"
data = load_arrow('data/01 download', 'BTCUSDT_30m')
data

,Time,Open,High,Low,Close,Volume,VolumeUSDT,TakerVolume,TakerVolumeUSDT,Trades
0,25049040.0,4261.48,4280.56,4261.32,4261.45,11.308926,4.822475e+04,3.936174,1.679304e+04,49
1,25049070.0,4280.00,4313.62,4267.99,4308.83,35.872083,1.541414e+05,31.224329,1.341594e+05,122
2,25049100.0,4308.83,4328.69,4304.31,4320.00,21.048648,9.086429e+04,19.396570,8.374608e+04,73
3,25049130.0,4320.00,4320.00,4291.37,4315.32,2.186268,9.440531e+03,2.051501,8.862196e+03,29
4,25049160.0,4330.29,4330.29,4309.37,4311.02,3.566277,1.540976e+04,2.302077,9.951673e+03,14
...,...,...,...,...,...,...,...,...,...,...
108850,28322790.0,34773.41,34840.88,34720.00,34822.47,640.848390,2.228644e+07,361.401370,1.256704e+07,24730
108851,28322820.0,34822.48,34837.93,34680.86,34748.76,582.823930,2.025871e+07,264.199690,9.183110e+06,25664
108852,28322850.0,34748.77,34748.77,34625.85,34721.23,827.433950,2.869384e+07,403.605720,1.399498e+07,34697
108853,28322880.0,34721.23,34739.25,34692.06,34739.25,337.937110,1.173263e+07,154.450700,5.362210e+06,19718


# II Limpieza de datos

Los datos del precio de mercado se generan en tiempo continuo por lo que no deberían existir "huecos" en los mismos. Sin embargo, por razones desconocidas, se identificaron algunos datos que no se encuentran registrados en los servidores de Binance. Esto podemos constatarlo con la variable **Time** ya que si existen saltos irregulares entre sus valores significaría que hay datos no registrados. Para este caso, en particular, los saltos de tiempo deben ser de 30 minutos:

In [4]:
index = np.where(data['Time'].diff() != 30)[0][1:]
print(f'Indices con saltos de tiempo:\n{index}')

empty = (np.array(data['Time'][index])-np.array(data['Time'][index-1]))/30
print(f'\nNúmero de datos faltantes en cada salto:\n{empty}')

print(f'\nTotal de datos faltantes:  {sum(empty)}')

print(
    f'\nLos datos faltantes representan un {round(100*sum(empty)/len(data),2)}% del los datos si registrados.')

Indices con saltos de tiempo:
[  985  5908  6704  8375 14935 14985 15293 20425 21658 27308 30370 31491
 34763 39067 39639 43283 43781 44437 46914 49981 57422 58444 58604 60910
 62008 64165 64405 69672 71929 97905]

Número de datos faltantes en cada salto:
[14.  3.  4. 68. 21.  4. 16.  8. 15. 13. 21.  2. 17.  5.  5.  3. 12.  5.
  6.  8.  3.  9.  3.  3.  4.  6.  9. 10.  5.  3.]

Total de datos faltantes:  305.0

Los datos faltantes representan un 0.28% del los datos si registrados.


Dicho lo anterior, los "huecos", que registran los datos, representan una proporción minúscula en comparación al tamaño total porque que resulta razonable estimar dicha información con base en los datos existentes. De manera general, para el caso de los precios, la estimación se realiza por medio de aproximaciones lineales entre los extremos de los datos existentes, en los casos de **High** y **Low** se añade, además de lo anterior, un factor de volatilidad proporcional a la cantidad de datos faltantes. Para estimar los volúmenes de operación se utiliza el promedio de los datos previos registrados con una ventana igual al tamaño de datos faltantes.

En el archivo `data/info/BLOCK_BTCUSDT_30m.csv` se encuentra una descripción de los bloques de datos que no registran información faltante, y la cantidad de datos que fueron estimados al final de cada bloque. Las bases de datos "limpias" fueron guardas en `data/02 clean` y les fue añadida la variable binaria **Filled** que indica si el renglón en cuestión fue estimado ($1$) o es original ($0$).



Por otro lado, en cuando a la calidad de datos existentes, se observa un comportamiento atípico en los primeros registros, causado por la baja cantidad de operaciones en el mercado, esto se debe a que la plataforma Binance comenzó a operar a mediados del año 2017 y nuestro primer registro de datos se ubica en la fecha 2017-08-16. Con el propósito de eliminar el sesgo causado por la falta de operaciones en el mercado, se opta por restringir una fecha mínima para el análisis, la fecha elegida es el 2018-02-12 la cual puede ser modificada, antes de realizar la transformación de los datos, en el archivo `src/python/util/parameters.json`.

<figure>
    <img src="img/logtrades.jpg"  width="60%" />
    <figcaption>Al inicio del periodo se observa un reducido nivel de operaciones.</figcaption>
</figure>

# III Transformación de datos

Después de realizar la limpieza de los datos, el script `test/02 transform.py` lee la información de la carpeta `data/02 clean`, realiza una serie de transformaciones y guarda los resultados en la carpeta `data/03 transform`. Las transformaciones aplicadas se dividen en tres grupos:

- **Logarítmicas:**  
Dada $X$ una serie de tiempo, definimos la transformación logarítmica como $\log\left(c+X\right)$, para alguna constante $c$. Esta transformación tiene el propósito de reducir el sesgo de las series y por lo tanto disminuir la cantidad de valores extremos. Las series obtenidas por este método fueron las siguientes:

    - **Open_Log** $=\log\left(\right.$**Open** $\left.\right)$
    - **High_Log** $=\log\left(\right.$**High**$\left.\right)$
    - **Low_Log** $=\log\left(\right.$**Low**$\left.\right)$
    - **Close_Log** $=\log\left(\right.$**Close**$\left.\right)$
    - **Volume_Log** $=\log\left(\right. 1\,+\,$**Volume**$\left.\right)$

- **Normalizadas (normalización móvil):**  
Dados $X$ una serie de tiempo, $m_X$ la estimación de su media a lo largo del tiempo y $st_X$ la estimación de su desviación estándar a lo largo del tiempo, definimos la normalización móvil de $X$ como $(X-m_X)/st_X$. Esta transformación pretende homogeneizar la variabilidad de los datos a lo largo del tiempo. Las series normalizadas son las siguientes:

    - **Open_Norm** $=\left(\right.$**Open**$\,-\,m_p\left.\right)/s_p$
    - **High_Norm** $=\left(\right.$**High**$\,-\,m_p\left.\right)/s_p$
    - **Low_Norm** $=\left(\right.$**Low**$\,-\,m_p\left.\right)/s_p$
    - **Close_Norm** $=\left(\right.$**Close**$\,-\,m_p\left.\right)/s_p$
    - **Volume_Norm** $=\left(\right.$**Volume**$\,-\,m_v\left.\right)/s_v$  

    **Nota**: Observe que para normalizar los precios se utilizan las mismas series de media y desviación estándar, esto permite que, para cada tiempo, los precios normalizados guarden las mismas proporciones con respecto a los precios originales.


- **Otras transformaciones:**

    - **Close_Rate**: Proporción entre dos precios de cierre consecutivos. Esta transformación puede ser útil para compensar las posibles pérdidas de información que puedan ocurrir al realizar la normalización móvil de la series de precios:  
    
        - **Close_Rate**[0]  $ = 100*\left(\right.$ **Close**[0] $/$ **Close**[1] $\,-\,1\left.\right)$  
        (la notación $X$[i] hace referencia a $X_{t+i}$  para cualquier $t$)
        $$$$
        
    - **Volume_Qty**: Cantidad de volumen en BTC con respecto al volumen en USDT. El objetivo de esta transformación es identificar qué moneda (BTC o USDT) es fue más preciada dentro del mercado, al final del intervalo de tiempo:

        - **Volume_Qty** $= 100 \,*$ **Volume** $/\left(\right.$ **Volume** $+$ **VolumeUSDT** $/$ **Close**$\left.\right)$
        $$$$
    - **Taker_Prop**: Porporción de volumen de BTC que fue comerciado por los tomadores de mercado, con respecto al volumen total de BTC comerciado en el intervalo de tiempo, puede ser un indicador de las preferencias de los participantes del mercado:
        - **Taker_Prop** $= 100 \,*$ **TakerVolume** $/$ **Volume**
        $$$$
    - **Volume_Trade**: Relación del volumen de BTC con respecto a número de operaciones, puede servir para identificar si los cambios en el volumen son generales o inducidos por datos atípicos:
        - **Volume_Trade** $= 100 \,*$ **Volume** $/$ **Trades**
        $$$$
    - **Volume_Trade_Norm**:  Volume_Trade normalizado. Se normaliza la variable **Volume_Trade** con fin de mejorar sus características:
        - **Volume_Trade_Norm** $=\left(\right.$**Volume_Trade**$\,-\,m_{vt}\left.\right)/s_{vt}$


# IV Sub-bases

En el directorio  `src.python.util.basic` se encuentra implementada la función `datasets`, que sirve para gestionar las bases de datos transformadas. Por defecto, la función devuelve las variables más relevantes del mercado (TOHLCV):

In [5]:
from src.python.util.basic import datasets

datasets(
    symbol='BTCUSDT',
    interval='30m'
)

,Time,Open,High,Low,Close,Volume
0,25306560.0,8063.82,8218.70,8053.00,8174.00,997.148934
1,25306590.0,8174.00,8270.42,8139.00,8198.42,869.892112
2,25306620.0,8198.42,8326.59,8154.68,8315.76,702.159269
3,25306650.0,8315.76,8417.00,8270.00,8345.51,1521.901408
4,25306680.0,8345.51,8425.00,8281.00,8411.09,799.721966
...,...,...,...,...,...,...
100541,28322790.0,34773.41,34840.88,34720.00,34822.47,640.848390
100542,28322820.0,34822.48,34837.93,34680.86,34748.76,582.823930
100543,28322850.0,34748.77,34748.77,34625.85,34721.23,827.433950
100544,28322880.0,34721.23,34739.25,34692.06,34739.25,337.937110


La función `datasets` se encarga principalmente de dividir la base principal en las sub-bases necesarias para el entrenamiento de la red neuronal. Además, permite estandarizar las bases añadidas.

In [6]:
training, validation = datasets(
    symbol='BTCUSDT',
    interval='30m',
    # Añadir variables de interés
    variables=[
        'Open_Norm', 'High_Norm', 'Low_Norm', 'Close_Norm',
        'Close_Rate', 'Volume_Log', 'Volume_Qty', 'Taker_Prop', 'Volume_Trade_Norm'
    ],
    # Elegir el tamaño de cada grupo de datos (El grupo entrenamiento siempre debe ser el primero)
    subsets=[79932, 19983],
    # Elegir el nivel de confianza de la estandarización (100% es la estandarización usual)
    conf_level=99,
    # Elegir isolated=False indica que la estandarización se realiza usando como referencia
    # los rangos del grupo de entrenamiento, en caso de que isolated=True cada grupo tendrá sus
    # rangos de estandarización independientes.
    isolated=False
)

               size    prop                time
A             79932   79.5% 2018-02-12 00:00:00
B             19983  19.87% 2022-09-04 06:00:00
not assigned    631   0.63% 2023-10-25 13:30:00


In [7]:
training

,Time,Open,High,Low,Close,Volume,Open_Norm,High_Norm,Low_Norm,Close_Norm,Close_Rate,Volume_Log,Volume_Qty,Volume_Trade_Norm
0,25306560.0,8063.82,8218.70,8053.00,8174.00,997.148934,0.371084,0.365223,0.381827,0.375413,0.819831,0.493547,0.708261,0.330032
1,25306590.0,8174.00,8270.42,8139.00,8198.42,869.892112,0.374967,0.367079,0.384856,0.376306,0.572511,0.461468,0.484329,0.314643
2,25306620.0,8198.42,8326.59,8154.68,8315.76,702.159269,0.375860,0.369092,0.385441,0.380433,0.835052,0.411149,0.981453,0.280502
3,25306650.0,8315.76,8417.00,8270.00,8345.51,1521.901408,0.379998,0.372308,0.389493,0.381514,0.586189,0.592915,0.460801,0.366675
4,25306680.0,8345.51,8425.00,8281.00,8411.09,799.721966,0.381081,0.372635,0.389919,0.383843,0.685423,0.441710,0.768900,0.281905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79927,27704370.0,19757.35,19777.87,19753.42,19772.79,1844.479460,0.397055,0.387153,0.407839,0.398437,0.521346,0.638104,0.519078,0.388902
79928,27704400.0,19772.81,19793.76,19758.92,19776.53,1857.330060,0.397863,0.387993,0.408044,0.398537,0.507638,0.639736,0.498051,0.369265
79929,27704430.0,19777.46,19812.19,19755.00,19801.74,1640.264920,0.398023,0.388994,0.407685,0.399940,0.532805,0.610521,0.541083,0.330409
79930,27704460.0,19801.74,19807.13,19761.43,19776.73,1834.299430,0.399371,0.388563,0.407949,0.398304,0.473973,0.636803,0.479182,0.349696


In [8]:
validation

,Time,Open,High,Low,Close,Volume,Open_Norm,High_Norm,Low_Norm,Close_Norm,Close_Rate,Volume_Log,Volume_Qty,Volume_Trade_Norm
79932,27704520.0,19768.94,19787.25,19730.24,19734.01,2134.78825,0.397121,0.387090,0.405800,0.395453,0.462256,0.672467,0.423690,0.361890
79933,27704550.0,19734.02,19739.81,19679.01,19700.96,3993.35414,0.394866,0.384059,0.402563,0.393314,0.464428,0.819719,0.478801,0.443802
79934,27704580.0,19700.96,19733.81,19675.49,19723.50,2936.85393,0.392731,0.383569,0.402224,0.394558,0.529776,0.747462,0.550683,0.412323
79935,27704610.0,19723.21,19728.76,19615.00,19632.40,3269.77861,0.393966,0.383140,0.398432,0.388900,0.396177,0.772711,0.357918,0.476635
79936,27704640.0,19630.95,19680.97,19583.10,19656.41,5140.94761,0.388224,0.380094,0.396372,0.390239,0.531605,0.879120,0.536620,0.474856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99910,28303860.0,34241.41,34446.42,34171.74,34252.91,1661.56436,1.048773,1.056057,1.052306,1.048934,0.511046,0.613554,0.444987,0.466475
99911,28303890.0,34252.91,34297.55,34170.05,34199.20,989.84454,1.048612,1.044208,1.051203,1.044062,0.466902,0.491819,0.475726,0.424863
99912,28303920.0,34199.20,34435.55,34199.19,34370.00,1173.32253,1.043767,1.053255,1.052348,1.055422,0.619032,0.531779,0.545560,0.402849
99913,28303950.0,34370.01,34417.36,34265.46,34337.01,993.38220,1.055110,1.050928,1.056136,1.052053,0.481001,0.492658,0.517566,0.443861


In [9]:
datasets(
    symbol='BTCUSDT',
    interval='30m',
    variables=[
        'Open_Norm', 'High_Norm', 'Low_Norm', 'Close_Norm',
        'Close_Rate', 'Volume_Log', 'Volume_Qty', 'Volume_Trade_Norm'
    ],
    subsets=[69932, 20983, 6998],
    conf_level=99
)

               size    prop                time
A             69932  69.99% 2018-02-12 00:00:00
B             20983   21.0% 2022-02-07 22:00:00
C              6998    7.0% 2023-04-21 01:30:00
not assigned   2002    2.0% 2023-09-13 20:30:00


(             Time      Open      High       Low     Close       Volume  \
 0      25306560.0   8063.82   8218.70   8053.00   8174.00   997.148934   
 1      25306590.0   8174.00   8270.42   8139.00   8198.42   869.892112   
 2      25306620.0   8198.42   8326.59   8154.68   8315.76   702.159269   
 3      25306650.0   8315.76   8417.00   8270.00   8345.51  1521.901408   
 4      25306680.0   8345.51   8425.00   8281.00   8411.09   799.721966   
 ...           ...       ...       ...       ...       ...          ...   
 69927  27404370.0  44417.04  44495.02  44215.46  44252.80   972.892350   
 69928  27404400.0  44252.31  44438.93  44093.93  44164.46   975.956800   
 69929  27404430.0  44164.46  44226.88  43792.77  44131.88  1264.707780   
 69930  27404460.0  44131.88  44248.71  43769.64  43858.67  1047.454280   
 69931  27404490.0  43854.52  44086.81  43850.00  44085.24   512.924920   
 
        Open_Norm  High_Norm  Low_Norm  Close_Norm  Close_Rate  Volume_Log  \
 0       0.376665   